In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
widgets.IntSlider(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

IntSlider(value=7, continuous_update=False, description='Test:', max=10)

In [4]:
data = {}

In [5]:
data_name = "Dummy"
raw = pd.read_csv(f"{data_name}.csv", sep=';', encoding='latin1').dropna()
raw.head()

,Core,AxialPosition,Location,SpectralIndex,C/E - 1 [%] and Uncertainty,Exp. Uncertainty [%],Library
0,A,Core Midplane,Lead Assembly,F28/F25,0.80,2,JEFF-3.3
1,A,Core Midplane,Lead Assembly,F49/F25,0.85,2,JEFF-3.3
2,A,Core Midplane,Lead Assembly,F28/F25,0.90,2,ENDF/B-VIII.0
3,A,Core Midplane,Lead Assembly,F49/F25,0.95,2,ENDF/B-VIII.0
4,A,Core Midplane,Lead Assembly,F28/F25,0.05,2,JEFF-3.3 std.


In [6]:
data["All"] = raw.copy()
data['No_IPS'] = raw.copy()
data['No_IPS'] = data['No_IPS'].query("not Location.str.startswith('IPS')", engine="python")
data['No_IPS'] = data['No_IPS'].query("Location != 'Fuel Midrange-void'", engine='python')

## Plot functions

In [7]:
def get_query(items):
    query_str = ""
#     query_str += "Instrument == '" + items['Instrument'] + "' and " if items["Instrument"] is not None and items["Instrument"] != 'all' else ""
    query_str += "Core == '" + items['Core'] + "' and " if items["Core"] is not None and items["Core"] != 'all' else ""
    query_str += "AxialPosition == '" + items['AxialPosition'] + "' and " if (items["AxialPosition"] is not None and items["AxialPosition"] != 'all') else ""
    query_str += "Location == '" + items['Location'] + "' and " if items["Location"] is not None and items["Location"] != 'all' else ""
#     query_str += "Coordinates == '" + items['Coordinates'] + "' and " if items["Coordinates"] is not None and items["Coordinates"] != 'all' else ""
#     query_str += "Material == '" + items['Material'] + "' and " if items["Material"] is not None and items["Material"] != 'all' else ""
    query_str += "SpectralIndex == '" + items['SpectralIndex'] + "' and " if items["SpectralIndex"] is not None and items["SpectralIndex"] != 'all' else ""
    query_str += "Library == '" + items['Library'] + "' and " if items["Library"] is not None and items["Library"] != 'all' else ""
#     query_str += "Code == '" + items['Code'] + "' and " if items["Code"] is not None and items["Code"] != 'all' else ""

    return query_str[:-5]

In [8]:
def get_order(hue):
    match hue:
        case 'Library':
            hue_order = ['ENDF/B-VIII.0', 'ENDF/B-VIII.0 std.', 'JEFF-3.3', 'JEFF-3.3 std.', 'JENDL-4.0', 'JENDL-4.0 std.']
        case 'Core':
            hue_order = ['CR0/2011', 'CR0/2014', 'CC5', 'CC6', 'CC7', 'CC8', 'CC9', 'CC12', 'CC13']
        case 'Instrument':
            hue_order = ['FC', 'AF']
        case 'AxialPosition':
            hue_order = ['Core Midplane', 'Core Top', 'Top Reflector', 'Top Reflector Midplane'] # are the last 2 just the same but labelled differently in CC13 (AF/FC)?
        case 'Coordinates':
            hue_order = ['1,1', '2,1', '2,2', '1,-3', '-3,1', '3,3', '4,1', '-4,1', '-5,1', '2,-5', '1,-6']
        case 'Location':
            hue_order = ['Fuel Center', 'Fuel Midrange', 'Fuel Midrange-void', 'Fuel Periphery', 'Lead Assembly', 'Graphite Reflector', 'Top Reflector Center',
                       'IPS-1', 'IPS-2', 'IPS-5', 'IPS-6', 'IPS-7', 'IPS-8']
        case 'Material':
            hue_order = ['U', 'Pb', 'C', 'PE', 'U+Al+PE+Cd', 'U+Al+air+Cd', 'Al+PE+Cd']
        case 'SpectralIndex':
            hue_order = ["F24/F25", "F28/F25", "F37/F25", "F49/F25", "F40/F25", "F42/F25"]
        case 'Code':
            hue_order = ['MCNP6.2', 'Serpent2']
        case None:
            hue_order = None 
    return hue_order

def get_palette(hue, order):
    match hue:
        case 'Library':
            sns.set_palette(sns.color_palette("Paired"))
            err = sum(['std.' in i for i in order])
            palette = ["C1", "C0", "C3", "C2", "C5", "C4", "C7", "C6", "C9", "C8", "C11", "C10"] if err else ["C1", "C3", "C5", "C7", "C9", "C11"]
        case _:
            sns.set_palette(sns.color_palette("husl", len(order)))
            palette = None
    return palette

In [9]:
def single_plot(#Instrument,
               Core,
               AxialPosition,
               Location,
#                Coordinates,
#                Material,
               SpectralIndex,
               Library,
#                Code,
               X, HUE, Y='C/E - 1 [%] and Uncertainty', data_key='AroundZero_and_Uncertainty',
               exp_err_bars=True, err=True, title=True, save=False, vertical_lines=True, symmetric=False, save_as=""):

    items = {#"Instrument": Instrument if Instrument != 'any' else None,
              "Core": Core if Core  != 'any' else None,
              "AxialPosition": AxialPosition if AxialPosition != 'any' else None,
              "Location": Location if Location != 'any' else None,
#               "Coordinates": Coordinates if Coordinates != 'any' else None,
#               "Material": Material if Material != 'any' else None,
              "SpectralIndex": SpectralIndex if SpectralIndex != 'any' else None,
              "Library": Library if Library != 'any' else None,
#               "Code": Code if Code != 'any' else None,
              }
    data_ = data[data_key].copy()
    subset = [k for k, v in items.items() if v is not None] + ["C/E - 1 [%] and Uncertainty"] + ["Exp. Uncertainty [%]"]
    
    query_str = get_query(items)
    
    if not err and Library != 'any':
        if Library != 'all':
            query_str += f" and Library != '{Library} std.'" 
        else:
            query_str += " and Library != 'ENDF/B-VIII.0 std.'"
            query_str += " and Library != 'JEFF-3.3 std.'"
            query_str += " and Library != 'JENDL-4.0 std.'"
    
    print(query_str)

    df = data_[subset].query(query_str)
    
    hue_order = [i for i in get_order(HUE) if i in df[HUE].unique()]
    palette = get_palette(HUE, hue_order)

    fig, ax = plt.subplots(figsize=(7, 3), dpi=300, sharex=True)
    try:
        sns.barplot(data=df,
                    x=X,
                    y=Y,
                    hue=HUE,
                    hue_order=hue_order,
                    palette=palette,
                    ax=ax
                    )
        plt.xticks(rotation=90)
        if exp_err_bars:
            ax.errorbar(data=df, x=X, y=df.shape[0] * [0], yerr='Exp. Uncertainty [%]', capsize=4, ecolor='k', color='gray', linewidth=0.7, label=None)
        
    except IndexError:
        raise Exception("Index Error: apparently there is no data corresponding to the selection")
    
    if title:
        ax.set_title(f'C/E of {SpectralIndex} fission rate' + 's' if SpectralIndex == 'all' else '')
    
    hlines = [0, -5, -10, -15, -20, -25, -30, -35, -40, -45, -50, -55, -60, -65, -70, -75, -80, -85, -90, -95, -100]
    if vertical_lines: ax.grid(which='major', axis='x', linestyle='--')
    for h in hlines:
        if df["C/E - 1 [%] and Uncertainty"].min() <= h:
            ax.axhline(h, alpha=0.5, color='grey')
            if symmetric: ax.axhline(-h, alpha=0.5, color='grey')
    
    lgd = plt.legend(bbox_to_anchor=(1, -.4), ncol=3, frameon=False)
    if save:
        fig.savefig('Plots\\' + save_as, bbox_extra_artists=(lgd,), bbox_inches='tight')
    return ax

## Interactive Plot

In [10]:
interact(single_plot,
         Core=np.append(['any', 'all'], raw.Core.unique()),
         AxialPosition=np.append(['any', 'all'], raw.AxialPosition.unique()),
         Location=np.append(['any', 'all'], raw.Location.unique()),
         SpectralIndex=np.append(['any', 'all'], raw.SpectralIndex.unique()),
         Library=np.append(['any', 'all'], raw.Library.unique()),
         Y=np.append([None], raw.columns),
         X=np.append([None], raw.columns),
         HUE=np.append([None], raw.columns),
         data_key=data.keys())

interactive(children=(Dropdown(description='Core', options=('any', 'all', 'A'), value='any'), Dropdown(descrip…

<function __main__.single_plot(Core, AxialPosition, Location, SpectralIndex, Library, X, HUE, Y='C/E - 1 [%] and Uncertainty', data_key='AroundZero_and_Uncertainty', exp_err_bars=True, err=True, title=True, save=False, vertical_lines=True, symmetric=False, save_as='')>

## VENUS-F configurations and measurements: brief description

* [CR0/2011](#chapter1)
* [CC5](#chapter3)
* [CC7](#chapter5)
* [CC8](#chapter6)
* [CC9](#chapter7)
* [CC10](#chapter8)
* [CC11](#chapter9)
* [CC12](#chapter10)
* [CC13](#chapter11)

### CR0 <a class="anchor" id="chapter1"></a>

Configuration loaded with metallic U fuel and Pb.

Measurements performed in E1, E2 and E3 in 2211 and in E2 and E3 in 2014.

![alternative text](DashboardImages/CR0.JPG)

![alternative text](DashboardImages/CR0Assembly.JPG)

![alternative text](DashboardImages/CR0Spectra.JPG)

### CC5 & CC6 <a class="anchor" id="chapter3"></a>

Configuration loaded with metallic U fuel, Pb snd Al2O3.
CC6 consists in the same lattice as CC5 where position (+1, -3) was surrounded by Al2O3 assemblies to represent ALFRED.

Measurements performed in (+1, +1) and (+1, -3) in CC5.
Measurements performed in (+1, -3) in CC6.

![alternative text](DashboardImages/CC5.JPG)

![alternative text](DashboardImages/CC5Assembly.JPG)

![alternative text](DashboardImages/CC5CC6pectra.JPG)

### CC7 <a class="anchor" id="chapter5"></a>

Configuration loaded with metallic U fuel, Pb snd Al2O3.

Measurements performed in (-5, +1).

![alternative text](DashboardImages/CC7.JPG)

![alternative text](DashboardImages/CC7Assembly.JPG)

### CC8 <a class="anchor" id="chapter6"></a>

Configuration loaded with metallic U fuel, Pb snd Al2O3.

Measurements performed in (+1, +1), (-3, 1), (4, 1), (-4, 1), (-5, 1).

![alternative text](DashboardImages/CC8.JPG)

![alternative text](DashboardImages/CC8Assembly.JPG)

![alternative text](DashboardImages/CC8Spectra.JPG)

### CC9 <a class="anchor" id="chapter7"></a>

Configuration loaded with metallic U fuel, Bi snd Al2O3.

Measurements performed in (+1, +1), (1, -3), (2, -5), (2, 1), (2, 1)-voided.

![alternative text](DashboardImages/CC9.JPG)

![alternative text](DashboardImages/CC9Spectra.JPG)

### CC10 <a class="anchor" id="chapter8"></a>

### CC11 <a class="anchor" id="chapter9"></a>

### CC12 <a class="anchor" id="chapter10"></a>

Configuration loaded with metallic U fuel, Bi snd Al2O3.

Measurements performed in (+1, +1), (1, -3), (2, -5), (2, 1), (2, 1)-voided.

![alternative text](DashboardImages/CC12.JPG)

![alternative text](DashboardImages/CC12-IPS.JPG)

### CC13 <a class="anchor" id="chapter11"></a>